In [1]:
import sys
import os 
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path: 
    sys.path.append(module_path)

import pandas as pd  
import numpy as np  

import tools.Sample_Tools as smpl
import tools.Pretreat_Tools as pretreat
from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)

# import ind.alpha191 as a191


import matplotlib.pyplot as plt
import seaborn as sns

from base.JuUnits import parallal_task,task_chunk_split
from base.JuUnits import excute_for_multidates



# from QUANTAXIS.QAUtil import DATABASE
# from QUANTAXIS.QAUtil import  trade_date_sse
# from QUANTAXIS.QAUtil.QADate_trade import (
#     QA_util_get_pre_trade_date,
#     QA_util_get_next_trade_date,
#     QA_util_if_tradetime
# )

# import inspect

# from sklearn import linear_model

%load_ext autoreload
%autoreload 2
%aimport ind.alpha191

In [2]:
###加载数据

ids = np.arange(1,192)
# np.random.shuffle(ids)
files = list(map(lambda x:'191_alpha'+'0'*(3- int(np.log10(x)+1))+str(x)+'_train',ids))

def load_alpha_factors(file_names):
    from tools.Cacher import (CACHE_TYPE, save_cache,load_cache_adv,load_cache)
    from base.JuUnits import excute_for_multidates
    import numpy as np
    import pandas as pd 
    
    temp = []
    for file in file_names:
        print(file)
        temp.append(load_cache(file, cache_type=CACHE_TYPE.FACTOR))
    
    return pd.concat(temp,axis=1)
        
worker=6
task = task_chunk_split(files, worker)
results = parallal_task(worker, load_alpha_factors, task)
res = pd.concat(results,axis=1)

Now in the main code. Process name is: base.JuUnits
base.JuUnits, subpid:6844  pid:25024


  0%|          | 0/6 [00:00<?, ?it/s]

In [3]:
def nan_count(x):
    return round(x.isna().sum(axis=0)/len(x),4)*100

def filter_invalid_factor(fators_df):
    ## 缺失30%以上，且3倍方差以上，废弃
    ## 加极端判断是为了保证留下那种大窗口期的因子。
    
    fators = fators_df.replace([np.inf, -np.inf], np.nan)
    
    #计算缺失率
    valid_rate = excute_for_multidates(fators, nan_count, level=1) 
    valid_distribute_boundary = valid_rate.mean(axis=0) + valid_rate.std(axis=0)*3
    #缺失率符合当前因子在所有标的上，符合3个标准差的
    filter_contition = valid_rate > valid_distribute_boundary 
    
    def filter_invalid(x):
        code = x.index.get_level_values(1)[0]
        distribute_condition = filter_contition.loc[code] 
        #符合3个标准差的列
        invalid_factors_name = distribute_condition[distribute_condition==True].index 
        #符合3个标准差，并且缺失30%以上的列
        thredhold_condition = valid_rate.loc[code][invalid_factors_name]>30 
        x[thredhold_condition.index] = np.nan 
        return x

    filtered = excute_for_multidates(fators, filter_invalid, level=1)
    
    return filtered

factors_filtered = filter_invalid_factor(res)

In [4]:
def filter_correlative_factor(fators_df, thredhold=.70, show_chart=False,verbose=False):
    #去掉相关性强的因子，相关性大于thredhold的因子（列）将被去除
    corr = fators_df.corr()
    matrix = corr.values
    #取得下三角矩阵
    matrix[np.triu_indices(matrix.shape[0])]=np.nan
    corr_triu = pd.DataFrame(matrix, columns=corr.columns, index=corr.index)

    if show_chart:
        plt.figure(figsize=(30,30))
        labels = corr.columns.str.replace('191_alpha','')
        ax = sns.heatmap(corr_triu, xticklabels=labels, yticklabels=labels, cmap="YlGnBu",cbar=True)

    # 取得符合条件的因子对后，再取“一对的左边”作为删除目标
    correlative_columns_name = corr_triu.abs().gt(thredhold).stack().loc[lambda x: x].index.get_level_values(0).unique().to_list()
    
    if verbose:
        print('delete correlative factors: {}'.format(','.join(correlative_columns_name)))
        
    correlative_droed = fators_df.drop(columns=correlative_columns_name)
    return correlative_droed

factors_filtered = filter_correlative_factor(factors_filtered,show_chart=False,verbose=True)

delete correlative factors: 191_alpha008,191_alpha014,191_alpha020,191_alpha022,191_alpha024,191_alpha026,191_alpha027,191_alpha028,191_alpha031,191_alpha034,191_alpha037,191_alpha040,191_alpha046,191_alpha047,191_alpha049,191_alpha050,191_alpha051,191_alpha052,191_alpha053,191_alpha055,191_alpha057,191_alpha058,191_alpha059,191_alpha062,191_alpha065,191_alpha066,191_alpha067,191_alpha068,191_alpha071,191_alpha072,191_alpha078,191_alpha079,191_alpha082,191_alpha088,191_alpha089,191_alpha090,191_alpha094,191_alpha095,191_alpha096,191_alpha097,191_alpha099,191_alpha100,191_alpha105,191_alpha106,191_alpha112,191_alpha116,191_alpha121,191_alpha122,191_alpha126,191_alpha128,191_alpha129,191_alpha132,191_alpha133,191_alpha134,191_alpha136,191_alpha137,191_alpha139,191_alpha143,191_alpha147,191_alpha150,191_alpha153,191_alpha154,191_alpha156,191_alpha160,191_alpha161,191_alpha162,191_alpha165,191_alpha167,191_alpha168,191_alpha173,191_alpha174,191_alpha175,191_alpha177,191_alpha180,191_alpha1

In [45]:
##标准化
# factors_filtered = (factors_filtered-factors_filtered.mean(axis=0,skipna=True))/  factors_filtered.std(axis=0,skipna=True)
# fenlei = factors_filtered.nunique()
# rows_count = factors_filtered.shape[0]
# if rows_count > 1000:
#     thredhold = rows_count *0.01
# else:
#     thredhold = 10
# is_label = fenlei<thredhold
# is_not_label = factors_filtered.columns.difference(is_label[is_label].index,sort=False)
# is_not_label_factors = factors_filtered[is_not_label]
# factors_filtered[is_not_label] = (is_not_label_factors-is_not_label_factors.mean(axis=0,skipna=True))/  is_not_label_factors.std(axis=0,skipna=True)


# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(factors_filtered.loc['2019-12-23'])
# factors_filtered['191_alpha075'].sort_values().unique()

0.12537998604160477

In [79]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):

factors = factors_filtered.iloc[:,0:10]# .loc[['2019-12-25']]

codes = factors.index.get_level_values(1).unique()
dates = factors.index.get_level_values(0).unique()
data_replenish = smpl.get_data(codes.to_list(),dates.min().strftime('%Y-%m-%d'),dates.max().strftime('%Y-%m-%d')).data
# data_replenish = smpl.get_data(codes.to_list(),'2019-12-25','2019-12-25').data
# ret = smpl.get_forward_return(data_replenish,'close')
ret = smpl.get_current_return(data_replenish,'close')
ret.name = 'ret'


mv_industry = smpl.add_marketvalue_industry(data_replenish).drop(columns=data_replenish.columns.to_list())
industy = mv_industry['industry']
cap = mv_industry['totalCapital']


# factors = factors.assign(cap=cap,industry = industy)
factors = factors.assign(country=1.0, cap=cap,industry = industy)
factors = pd.get_dummies(factors,'industry')

## 国家因子换到第一列
columns_ = factors.columns.to_list()
columns_.insert(0,columns_.pop(columns_.index('country')))
factors = factors.reindex(columns=columns_)



# adj_weight = np.sqrt(cap)/np.sqrt(cap).sum()
# adj_weight_mat = pd.DataFrame(np.diag(adj_weight),index=adj_weight.index, columns=adj_weight.index)

# del factors['country']


In [80]:


def multi_factors_model(factors_df,ret_series):
    ##factors_df 为单日，或多日混合，无date index。
    _invalid_check = nan_count(factors_df)
    ## 去除缺失5%以上的因子
    _invalid_checked = factors_df.drop(columns=_invalid_check[_invalid_check>5].index)
    ## 去除仍有少量缺失的标的
    _invalid_checked = _invalid_checked.dropna()
    ## 去除内容全部相同的列    
    _invalid_checked = _invalid_checked.loc[:, (_invalid_checked != _invalid_checked.iloc[0]).any()]
    ## 如果有国家因子需要加回来
    if 'country' in factors_df.columns:
        _invalid_checked = _invalid_checked.assign(country=1.0)
        ## 国家因子换到第一列
        _columns = _invalid_checked.columns.to_list()
        _columns.insert(0,_columns.pop(_columns.index('country')))
        _invalid_checked = _invalid_checked.reindex(columns=_columns)
        
    ## 去除内容列后，从新计算剩余的行业数量
    industy_count = _invalid_checked.columns.str.contains('industry_').sum()
    
    

    _cap=_invalid_checked['cap']

    #计算个股权重矩阵
    ##个股权重矩阵，权重为流通市值平方占比
    cap_temp = np.sqrt(_cap)
    wight = cap_temp/cap_temp.sum()
    V = np.diag(wight) #pd.DataFrame(np.diag(wight),columns = _cap.index,index = _cap.index)

    _industy_dummy = _invalid_checked.iloc[:,-1*industy_count:]
    industry_cap = _cap.T.dot(_industy_dummy)
    industry_wight = industry_cap/industry_cap.sum()

    X = _invalid_checked[_invalid_checked.columns.difference(['cap'],sort=False)]

    ##计算约束矩阵防止共线性问题
    #构建约束矩阵R，R为K*K-1矩阵，K为因子数量
    #将最后一个行业因子表示为其他行业的线性形式
    #移除最后一个行业所在的列，并对其所在的行进行权重处理
    k = X.shape[1] #总因子数量
    p = industy_count #行业因子数量
    # q = k-1-p #风格因子数量
    q = k-p #原风格因子数量

    diag_R = np.diag(np.ones(k))
    location = q
    R = np.delete(diag_R,location+p-1,axis = 1)
    adj_industry_weights = -industry_wight.div(industry_wight[-1]).iloc[:-1]
    R[location+p-1,location:location+p] = adj_industry_weights


    # showR = pd.DataFrame(R,columns=np.delete(X.columns,location+p-1),index=X.columns)
    # with pd.option_context("display.max_rows", None, "display.max_columns", None):
    #     display(showR)
    # showR.to_excel('./showR.xlsx')

    #求解投资组合股票权重矩阵
    Y = ret_series[X.index]
    W = R.dot(np.linalg.inv(np.array(R.T.dot(X.T).dot(V).dot(X).dot(R),dtype='float'))).dot(R.T).dot(X.T).dot(V)
    W = pd.DataFrame(W,index = X.columns,columns = X.index).round(6)
 
    #计算因子收益率
    f = W.dot(Y).T
    with pd.option_context("display.max_rows", None, "display.max_columns", None):
        display(W.dot(X).round(4))
    residual = Y - X.dot(f.T)
    residual.name='residual'
    # residual.sum()
    return residual

d='2019-12-25'
re = multi_factors_model(factors.loc[d],ret.loc[d])
re
# multi_factors_model(factors,ret)


,country,191_alpha001,191_alpha002,191_alpha003,191_alpha004,191_alpha005,191_alpha006,191_alpha007,191_alpha009,191_alpha010,191_alpha011,industry_PCB,industry_中药,industry_乳制品,industry_云软件服务,industry_产联网信息服务,industry_住宅开发,industry_保险,industry_光伏电池组件,industry_光伏硅料,industry_光伏硅片,industry_光伏辅材,industry_其他IT设备,industry_其他专业工程,industry_其他广告营销,industry_其他建材,industry_其他酒类,industry_军工电子,industry_冰洗,industry_功率半导体,industry_动力煤,industry_化学制剂,industry_化学工程,industry_医疗研发外包,industry_医院,industry_半导体封测,industry_半导体设备,industry_厨房小家电,industry_品牌消费电子,industry_商业物业经营,industry_商用载货车,industry_啤酒,industry_国有大型银行,industry_城商行,industry_基建工程,industry_安防设备,industry_家具,industry_家电零部件,industry_工程机械整机,industry_工程机械零件,industry_底盘与发动机系统,industry_快递,industry_房屋建设,industry_新兴软件,industry_旅游服务,industry_有机化工原料,industry_有机硅,industry_机场,industry_板材,industry_核力发电,industry_氨纶,industry_氯碱,industry_水产饲料,industry_水力发电,industry_水泥制造,industry_汽车电子电气系统,industry_油气开采,industry_油田服务,industry_消费电子组件,industry_涤纶,industry_清洁小家电,industry_港口,industry_游戏,industry_激光设备,industry_火力发电,industry_炼油化工,industry_煤化工,industry_特钢,industry_玻纤制造,industry_生猪养殖,industry_电信运营服务,industry_电动乘用车,industry_电池化学品,industry_电网自动化,industry_畜禽饲料,industry_白酒,industry_稀土,industry_空调,industry_系统设备,industry_纺织化学品,industry_综合乘用车,industry_聚氨酯,industry_肉制品,industry_股份制银行,industry_航天装备,industry_航海装备,industry_航空装备,industry_航空运输,industry_航运,industry_血液制品,industry_行业应用软件,industry_证券,industry_诊断服务,industry_调味品,industry_车身附件及饰件,industry_轨交设备,industry_轮胎轮毂,industry_配电设备,industry_钨钼,industry_钴,industry_钾肥,industry_铁路运输,industry_铜,industry_铝,industry_锂,industry_锂电池,industry_防水材料,industry_集成电路设计,industry_面板,industry_风电整机,industry_黄金
country,1.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,0.0,-0.0,-0.0,0.0049,0.0088,0.0059,0.0035,0.0047,0.0286,0.1068,0.0005,0.0017,0.0043,0.0008,0.0046,0.0018,0.0031,0.0013,0.0004,0.002,0.0041,0.0042,0.0177,0.0141,0.001,0.001,0.001,0.0012,0.0015,0.0019,0.0018,0.001,0.0005,0.003,0.139,0.0166,0.0156,0.0117,0.0015,0.0015,0.0075,0.0014,0.0039,0.0077,0.0099,0.0024,0.0055,0.0005,0.0009,0.0048,0.004,0.0025,0.0009,0.0008,0.0017,0.0025,0.0091,0.0014,0.034,0.0029,0.021,0.0044,0.0004,0.0043,0.0043,0.0014,0.0015,0.023,0.0032,0.0022,0.0012,0.0055,0.0059,0.0041,0.0028,0.0033,0.0025,0.0745,0.0012,0.0254,0.0046,0.0014,0.008,0.0055,0.0031,0.0959,0.0015,0.0048,0.0043,0.0112,0.002,0.0031,0.0032,0.0631,0.0007,0.0093,0.0044,0.0065,0.0009,0.0018,0.0029,0.0013,0.0012,0.0039,0.0052,0.0027,0.0014,0.0005,0.0012,0.0109,0.0073,0.0016,0.0041
191_alpha001,0.0,1.0,-0.0,0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,-0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000
191_alpha002,0.0,0.0,1.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0,-0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,-0.0000,0.0000,0.000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,-0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.0000,0.0000,0.0000,0.0000,-0.000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,-0.0000,0.000,-0.0000,0.0000,0.0000,0.000

code
000001    3.482905e-04
000002    7.177949e-03
000063    1.721391e-07
000066   -3.022069e-03
000069    4.105342e-03
              ...     
603806   -1.254873e-07
603833   -2.221089e-07
603882    5.144317e-08
603986    1.415844e-02
603993    2.555267e-07
Name: residual, Length: 211, dtype: float64

In [81]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(X.isna().sum(axis=0)>0)
# test
# test_invalid_check = nan_count(test)
# test_invalid_checked = test.drop(columns=test_invalid_check[test_invalid_check>5].index)
# test_invalid_checked.dropna()

# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     # display(pd.DataFrame(R.T.dot(X.T),columns=X.index,index=showR.columns).sum(axis=1))
#     display(pd.DataFrame(R.T.dot(X.T).dot(V).dot(X).dot(R),columns=showR.columns,index=showR.columns))
    
    # display(X.sum(axis=0))
    # display(R.T.dot(X.T))
# W

# W

NameError: name 'W' is not defined

In [77]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
#     display(f)

def factor_test(date_):
    ## 外部 factors，ret
    test_factor = factors.loc[date_]
    test_ret = ret.loc[date_]

    residual = multi_factors_model(test_factor,test_ret)

    retcurrent_index = retindex_all.index(pd.Timestamp(date_))
    retdate_range = retindex_all[retcurrent_index+1:retcurrent_index+4]
    
    
    return pd.DataFrame(residual)
    # residual_df = pd.DataFrame(residual.abs())
    # try:
    #     residual_df['ret3'] = residual_df.apply(lambda x:ret.loc[(retdate_range,x.name)].sum(),axis=1)
    # except Exception as e:
    #     print(date_,e.__class__.__name__,e)
    #     # traceback.print_exc()
    #     return None
    # return residual_df.sort_values(by='residual',axis='index', ascending=False)['ret3']
    


date_start = '2019-01-07'
step = 1
retindex_all = ret.index.get_level_values(0).unique().to_list()[0::step]
factors_date_all = factors.index.get_level_values(0).unique().to_list()[0::step]

test_date_index = factors_date_all.index(pd.Timestamp(date_start))
test_date_range = factors_date_all[test_date_index+1:test_date_index+int(710/step-1)] #710
# test_date_range


# res_cum_pos = pd.Series(0,index=factors.index.get_level_values(1).unique())
# res_cum_neg = pd.Series(0,index=factors.index.get_level_values(1).unique())
res_cum = []
for date_ in test_date_range:
    res = factor_test(date_)
    if not res is None:
    #     res_ = res.dropna()
    #     topN = res_[0:1]
    #     tailN = res_[-1:]
    #     res_cum_pos[topN.index] = res_cum_pos[topN.index]+topN
    #     res_cum_neg[tailN.index] = res_cum_neg[tailN.index]+tailN
        res_ = res.dropna()
        res_cum.append(res_.mean())
        if res_.mean().abs()[0] > 1:
            print(date_,res_.mean())
# print(np.sum(pos),np.sum(neg))


2019-06-05 00:00:00 residual    1.356904
dtype: float64
2019-07-12 00:00:00 residual    1.278271
dtype: float64
2019-09-25 00:00:00 residual    1.696281
dtype: float64
2019-11-28 00:00:00 residual   -1.193953
dtype: float64
2020-04-10 00:00:00 residual    2.513933
dtype: float64
2020-07-09 00:00:00 residual   -2.062287
dtype: float64
2021-03-18 00:00:00 residual   -1.159144
dtype: float64
2021-07-14 00:00:00 residual   -1.096776
dtype: float64
2021-08-10 00:00:00 residual    1.59567
dtype: float64


In [78]:
# with pd.option_context("display.max_rows", None, "display.max_columns", None):
# #     # display(X.dot(f.T))
# #     # display(residual.sort_values(ascending=1))
#     display(res_cum)

# res_cum_pos.mean(),res_cum_neg.mean()

a=np.array(res_cum).reshape(-1)
a.sort()
print(a.mean(), a.std()) #0.007843376600751448 0.6770983120962308

0.005935451783332656 0.2086955443641455


In [347]:
# bench=smpl.get_index_data('399300',start=str(test_date_range[0])[0:10],end=str(test_date_range[-1])[0:10])
# bench_close = bench.data.close
# (bench_close[-1]-bench_close[0])/bench_close[0]
# bench.data


# factors.loc[['2019-01-07','2019-01-08']].reset_index('date',drop=True).sort_index()

# tt_f = factors.loc[b1]
# tt_r = ret.loc[b1]

# # with pd.option_context("display.max_rows", None, "display.max_columns", None):399  195 197
#     # display(multi_factors_model(tt_f,tt_r))

# re = multi_factors_model(tt_f,tt_r)
re.loc[(slice(None),'000333')]

date
2019-06-04   -0.016917
2019-06-05   -0.023718
2019-06-06   -0.002573
2019-06-10    0.015342
2019-06-11   -0.008158
2019-06-12   -0.018935
2019-06-13   -0.018437
2019-06-14   -0.004742
2019-06-17   -0.006759
2019-06-18    0.006363
2019-06-19   -0.002109
2019-06-20   -0.027056
2019-06-21    0.007363
2019-06-24   -0.008441
2019-06-25   -0.001065
2019-06-26   -0.009351
2019-06-27   -0.016405
2019-06-28    0.021421
2019-07-01    0.011945
2019-07-02   -0.009109
Name: residual, dtype: float64

In [348]:
# a1=factors_date_all.index(pd.Timestamp('2019-06-04'))
# b1=factors_date_all[a1:a1+20]
with pd.option_context("display.max_rows", None, "display.max_columns", None):
    display(ret.loc[['2019-06-27','2019-06-28','2019-07-01','2019-07-02','2019-07-03','2019-07-04','2019-07-05']].loc[(slice(None),'000333')])


date
2019-06-27    0.001738
2019-06-28    0.043000
2019-07-01    0.013311
2019-07-02   -0.013136
2019-07-03    0.005731
2019-07-04    0.015993
2019-07-05   -0.015922
Name: ret_forward, dtype: float64

In [346]:
re.index.get_level_values(1).unique()

Index(['000001', '000002', '000063', '000066', '000069', '000100', '000157',
       '000166', '000301', '000333',
       ...
       '600011', '600015', '600016', '601229', '601328', '601818', '601009',
       '603486', '601138', '002049'],
      dtype='object', name='code', length=220)